In [1]:
# 가상환경 실행 : .\.venv\Scripts\activate.ps1

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
#from langchain_community.llms import Ollama
#from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
#from langchain_ollama import OllamaEmbeddings
import os
from langgraph.prebuilt import ToolNode
from typing import Literal
#from langgraph.graph import END
from langgraph.graph import START, END
from langgraph.graph import MessagesState, StateGraph
from langchain_core.prompts import PromptTemplate


In [2]:
# model cell

embeddings = OllamaEmbeddings(
    model="bge-m3"
)

llm = ChatOllama(
    model="qwen3:4b"
)

C:\Users\worb1\AppData\Local\Temp\ipykernel_1320\3511307189.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [4]:
persist_directory = "./chroma_db"
collection_name = 'requirements_list'
if os.path.exists('./chroma_db') and len(os.listdir('./chroma_db')) > 0:
    #기존 벡터 DB 가 존재할 경우.
    print("Vector DB 존재. 불러오기 시작.")
    
    vector_store = Chroma(
        embedding_function=embeddings,
        collection_name = collection_name,
        persist_directory = persist_directory
    )
    print("Vector DB 불러오기 완료.")
else:
    print("Vector DB 부재. 생성 시작.")
    # 1. 문서 로드
    loader = TextLoader("./docs/RFP_requirements.md", encoding="utf-8")
    documents = loader.load()

    # 2. 문서 나누기
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(documents)
    print(f"Chroma DB에 {len(splits)}개의 문서를 임베딩하여 저장 완료.")

    # 3. 벡터 스토어 생성
    vector_store = Chroma.from_documents(
        documents=splits, 
        embedding=embeddings, 
        persist_directory=persist_directory,
        collection_name=collection_name    
    )
    print("Vector DB 생성 완료.")


Vector DB 부재. 생성 시작.
Chroma DB에 41개의 문서를 임베딩하여 저장 완료.
Vector DB 생성 완료.


In [6]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

#query = "교육과정을 개설하는 기능은?"
#result = retriever.invoke(query)
#print(f"✅ 검색 결과: {result}")

In [7]:
from langchain_core.output_parsers import StrOutputParser
# 1. 소스코드 구현 내용 을 자연어로 변환 시키는 부분

code_interpreter_template = """
당신은 Code Review 전문가입니다.

지금부터 제공해주는 코드 파일들의 기능을 분석하세요.
분석 결과는 **아래의 출력 형식만** 따르세요.  
그 외의 어떤 설명, 영어 문장, `<think>` 등은 **절대 포함하지 마세요.**

information List : {information}

반드시 지켜야 할 규칙:
- 출력은 아래 형식만 사용합니다.
- 생각, 분석, 설명, 영어 문장은 절대 출력하지 마세요.
- `<think>`나 내부 분석 과정도 출력하지 마세요.
- 아래 형식처럼, 각 파일명과 기능을 한글로 요약해서 출력만 하세요.

출력 예시:
<output>
- A.java  
-- 사용자 등록 로직 구현.  
-- 사용자 삭제 로직 구현.  
-- 사용자 중복 여부 검사 로직 구현.  
</output>

이제 파일 목록을 분석한 후, 위 형식에 맞춰 출력만 하세요.
"""
code_interpreter_prompt = PromptTemplate.from_template(code_interpreter_template)
code_interpreter_result_chain = code_interpreter_prompt | llm | StrOutputParser()

code_information = [{
    "fileName" : "LoginService.java",
    "language": "java",
    "code" : """
        @Service
        public class LoginService {

            private final UserRepository userRepository;

            public LoginService(UserRepository userRepository) {
                this.userRepository = userRepository;
            }

            public boolean login(String email, String password) {
                return userRepository.findByEmail(email)
                        .map(user -> user.getPassword().equals(password))
                        .orElse(false);
            }
        }
    """}
    , 
    { 
    "fileName" : "mcp_server.py",
    "language": "python",
    "code" : """
        # 가상환경 실행 : .\.venv\Scripts\activate.ps1

        from mcp.server.fastmcp import FastMCP

        mcp = FastMCP(\"ppm\")

        @mcp.tool()
        def add(a: int, b: int) -> int:
            \"\"\"두 숫자를 더합니다.\"\"\"
            return a + b

        # dev : mcp dev ./fastmcp-server/mcp_server.py
        # prd : python ./fastmcp-server/mcp_server.py 
        if __name__ == \"__main__\":
            mcp.run(transport="stdio")
    """}]

result = code_interpreter_result_chain.invoke({"information": code_information})




# 2. 변환시킨 자연어를 retriever 에 넣는 부분



<>:57: SyntaxWarning: invalid escape sequence '\.'
<>:57: SyntaxWarning: invalid escape sequence '\.'
C:\Users\worb1\AppData\Local\Temp\ipykernel_1320\2134439076.py:57: SyntaxWarning: invalid escape sequence '\.'
  # 가상환경 실행 : .\.venv\Scripts\activate.ps1
